In [1]:
# 1
import spacy
import json
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

In [2]:
# 2
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Function to add custom rules based on the synonyms in JSON
def add_custom_rules():
    with open("OCR_raw_samples/X1.json", "r") as file:
        data = json.load(file)
        patterns = []
        for entry in data:
            for synonym in entry["Synonyms"]:
                patterns.append({"label": "TEST", "pattern": synonym})
        # print(patterns)
        ruler.add_patterns(patterns)

# Load custom rules
add_custom_rules()

In [ ]:
# print custom rules
print(ruler.patterns)

In [ ]:
# matcher = Matcher(nlp.vocab)

# def add_matcher_patterns():
#     # Define the pattern for matching units
#     pattern = [{"LOWER": {"IN": ["iu", "g", "mmol", "u"]}}, {"TEXT": "/"}, {"LOWER": {"IN": ["ml", "l"]}}]
#     matcher.add("unit", [pattern])

# # Add the matcher patterns
# add_matcher_patterns()

In [3]:
def add_matcher_patterns(matcher):
    # Define the pattern for matching units
    pattern = [{"LOWER": {"IN": ["iu", "g", "mmol", "u"]}}, {"TEXT": "/"}, {"LOWER": {"IN": ["ml", "l"]}}]
    matcher.add("unit", [pattern])

In [3]:
# 3
file_path = "OCR_raw_samples/0b8706dc-c9af-4c6b-887d-2f85b5a511e7.txt"

with open(file_path, 'r') as file:
    # text = file.read()
    data = file.read().replace("\n", " ")
    # text = clean_text(text)
doc = nlp(data)

In [5]:
# Initialize the matcher with the vocab
matcher = Matcher(nlp.vocab)
add_matcher_patterns(matcher)

# Dictionary to store the drug data
drug_data = {}

# Assume `doc` has been processed by `nlp(text)`
for ent in doc.ents:
    if ent.label_ == "TEST":
        if ent.text not in drug_data:
            drug_data[ent.text] = {"values": [], "units": []}
        current_parameter = ent.text

    elif ent.label_ == "CARDINAL" and "current_parameter" in locals():
        drug_data[current_parameter]["values"].append(ent.text)

# Process the entire document to find matches for units using the matcher
matches = matcher(doc)
for match_id, start, end in matches:
    matched_span = doc[start:end].text
    if "current_parameter" in locals():
        drug_data[current_parameter]["units"].append(matched_span)

# To display the organized drug data
print(drug_data)

{'Erythrocytes': {'values': ['14', '5', '0', '5'], 'units': []}, 'IgG': {'values': ['30/01/2024', '5', '30/01/2024', '31/01/24', '30/01/2024', '5', '30/01/2024'], 'units': []}, 'Sodium': {'values': ['140', '139', '140', '135', '140', '139', '140', '135', '139', '140', '138', '135'], 'units': []}, 'Potassium': {'values': ['5.5', '4.5', '3.5-5.5', '106', '106', '107', '106', '95', '5.5', '4.5', '3.5-5.5', '106', '106', '107', '106', '95', '4.5', '3.5-5.5', '107', '106', '108', '95'], 'units': []}, 'Bicarbonate': {'values': ['23', '23', '21'], 'units': []}, 'Urea': {'values': ['4.0', '4.8', '5.1', '2.5-7.0', '4.0', '4.8', '5.1', '2.5-7.0', '5.3', '5.2', '2.5-7.0'], 'units': []}, 'Creatinine': {'values': ['4.4', '1.007', '1.005'], 'units': []}, 'eGFR': {'values': ['90', '90', '90', '59', '2.15-2.55', '2.34', '2.15-2.55', '0.84', '30/01/2024', '24/09/18', '28/07/23', '31/01/24', '4.5', '4.7', '3.6-6.0', '4.5', '3.6-7.8', '3.6-6.0', '90', '90', '90', '59', '2.15-2.55', '2.34', '2.15-2.55', '

In [ ]:
# from spacy.matcher import Matcher

# matcher = Matcher(nlp.vocab)

# def extract_unit(doc_nlp):
#     # create pattern for unit extraction IU/mL, g/L,mmol/L, U/L
#     pattern = [{"LOWER": {"IN": ["iu", "g", "mmol", "u"]}}, {"TEXT": "/"}, {"LOWER": {"IN": ["ml", "l"]}}]
#     matcher.add("unit", [pattern])
#     matches = matcher(doc_nlp)
#     for match_id, start, end in matches:
#         span = doc[start:end]
#         print(span.text)

# extract_unit(doc)

In [4]:
# 4
# Initialize a dictionary to store drug parameters, values, and units
drug_data = {}

for ent in doc.ents:
    if ent.label_ == "TEST":
        # Start a new entry for this parameter if it doesn't exist
        if ent.text not in drug_data:
            drug_data[ent.text] = {"values": [], "units": []}
        # Focus on this parameter for subsequent values and units
        current_parameter = ent.text

    elif ent.label_ == "CARDINAL" and "current_parameter" in locals():
        # Append value to the current parameter
        drug_data[current_parameter]["values"].append(ent.text)

    elif ent.label_ == "ORG" and "current_parameter" in locals():
        # Append unit to the current parameter
        drug_data[current_parameter]["units"].append(ent.text)

# To display the organized drug data
print(drug_data)



{'Erythrocytes': {'values': ['14', '5', '0', '5'], 'units': ['UTI', 'MSU']}, 'IgG': {'values': ['30/01/2024', '5', '30/01/2024', '31/01/24', '30/01/2024', '5', '30/01/2024'], 'units': ['Rubella Serology Rubella', 'CMIA', 'IU/mL  Comment on Lab', 'HB-Sero', 'BIOCHEMISTRY  Date', 'Units Reference Status Fasting Fasting  ', 'Rubella Serology Rubella', 'CMIA', 'IU/mL  Comment on Lab']}, 'Sodium': {'values': ['140', '139', '140', '135', '140', '139', '140', '135', '139', '140', '138', '135'], 'units': []}, 'Potassium': {'values': ['5.5', '4.5', '3.5-5.5', '106', '106', '107', '106', '95', '5.5', '4.5', '3.5-5.5', '106', '106', '107', '106', '95', '4.5', '3.5-5.5', '107', '106', '108', '95'], 'units': []}, 'Bicarbonate': {'values': ['23', '23', '21'], 'units': []}, 'Urea': {'values': ['4.0', '4.8', '5.1', '2.5-7.0', '4.0', '4.8', '5.1', '2.5-7.0', '5.3', '5.2', '2.5-7.0'], 'units': []}, 'Creatinine': {'values': ['4.4', '1.007', '1.005'], 'units': ['MSU', 'Urine  pH 6 Protein Nil Glucose Nil 

In [ ]:
# 5


In [ ]:
result = []
drug, value, unit = {}, [], []
for ent in doc.ents:
    # print(ent.text, ent.label_)
    # print("-------------------")
    # check if the entity is TEST and entity is an ORG
    if ent.label_ == "TEST":
        # print(ent.text, ent.label_)
        drug["parameter"] = ent.text
        # print(result)
        # print("-------------------")

    if ent.label_ == "CARDINAL":
        # print(ent.text, ent.label_)
        # check if result has "parameter" key and add the value to the list
        if "parameter" in drug:
            value.append(ent.text)
        drug["value"] = value
        # save in a list
        # result["value"] = ent.text
        # print(result)
        # print("-------------------")

    if ent.label_ == "ORG":
        # print(ent.text, ent.label_)
        # check if result has "parameter" key and "value" key and add the unit
        if "parameter" in drug and "value" in drug:
            unit.append(ent.text)
        drug["unit"] = unit
        # print(result)
        # print("-------------------")
    
    print(drug)
    
    #     # get next token 
    #     # print(doc[ent.end].nbor())
    #     # print(doc[ent.end])
    #     next_token = doc[ent.end].nbor()
    #     if next_token.like_num:
    #         print(ent.text, next_token)
# print(result)